# Import

In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Initialisation

In [2]:
# First image should be the reference image
srcs = ["bouteilleetverre", "troisbouteilles", "unebouteille"]
imgs = [] # Liste of images
bws = [] # Liste of image in black and white
for src in srcs:
    img = cv2.imread(src + ".jpg")
    (thresh, bw) = cv2.threshold(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), 127, 255, cv2.THRESH_BINARY)
    # print(thresh)
    imgs.append(img)
    bws.append(bw)
    
# print(len(imgs))

# Base image and image in black and white

In [3]:
t = 0
for i in imgs:
    cv2.imshow("image" + str(t), i)
    t += 1
   
t = 0
for bwi in bws:
    cv2.imshow("bw" + str(t), bwi)
    t += 1

cv2.waitKey(0)

-1

# Canny Filter

In [4]:
t = 0
for i in imgs:
    img_canny = cv2.Canny(imgs[t], 200, 200)
    cv2.imshow("canny" + str(t), img_canny)
    t += 1
cv2.waitKey(0)

-1

# Prewitt Filter

In [5]:
kernelx = np.array([[1,1,1],[0,0,0],[-1,-1,-1]])
kernely = np.array([[-1,0,1],[-1,0,1],[-1,0,1]])

In [6]:
t = 0
for bw in bws:
    img_prewittx = cv2.filter2D(bw, -1, kernelx)
    img_prewitty = cv2.filter2D(bw, -1, kernely)

    cv2.imshow("prewittx"+str(t), img_prewittx)
    cv2.imshow("prewitty"+str(t), img_prewitty)
    cv2.imshow("prewittx+y"+str(t), img_prewittx+img_prewitty)
    t += 1
    
cv2.waitKey(0)

-1

# Sobel Filter

In [7]:
t = 0
for bw in bws:
    img_sobelx = cv2.Sobel(bw,cv2.CV_8U,1,0,ksize=5)
    img_sobely = cv2.Sobel(bw,cv2.CV_8U,0,1,ksize=5)
    img_sobel = img_sobelx + img_sobely

    cv2.imshow("sobel"+str(t), img_sobel)
    t += 1
    
cv2.waitKey(0)

-1

# Laplacien Filter

In [8]:
t = 0
for bw in bws:
    img_laplacian = cv2.Laplacian(bw, cv2.CV_64F)

    cv2.imshow("laplacian"+str(t), img_laplacian)
    t += 1
    
cv2.waitKey(0)

-1

# New Test

In [3]:
new_img = cv2.imread(srcs[0] + ".jpg")
gray = cv2.cvtColor(new_img, cv2.COLOR_BGR2GRAY)
(new_thresh, new_bw) = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

cv2.imshow("bw", new_bw)
cv2.waitKey(0)

-1

# Morphology

Trying to work with morphology operation for better shape

## Opening

In [4]:
kernel = np.ones((3,3), np.uint8)
opening = cv2.morphologyEx(new_bw, cv2.MORPH_OPEN, kernel, iterations = 2)

## Dilate

In [5]:
sure_bg = cv2.dilate(opening,kernel,iterations=3)

## Distance

In [6]:
dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
ret, sure_fg = cv2.threshold(dist_transform,0.7*dist_transform.max(),255,0)

# Show

In [7]:
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg,sure_fg)

cv2.imshow("opening", opening)
cv2.imshow("bg", sure_bg)
cv2.imshow("fg", sure_fg)
cv2.imshow("unknown", unknown)

cv2.waitKey(0)

-1

## Marking

In [8]:
ret, markers = cv2.connectedComponents(sure_fg)
markers = markers+1
markers[unknown==255] = 0

## Watershed

In [9]:
markers = cv2.watershed(new_img,markers)
new_img[markers == -1] = [255,0,0]

## Results

In [10]:
cv2.imshow("new_img", new_img)
cv2.waitKey(0)

-1

# Shape Matching

## First image

In [ ]:
contoursBase, hierarchyBase = cv2.findContours(sure_bg,
                                       cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
print("Number of Contours found = " + str(len(contoursBase)))
img = cv2.imread(srcs[0] + ".jpg")

cv2.drawContours(img, contoursBase, -1, (0, 255, 0), 3)

cv2.imshow('Contours', img)
cv2.waitKey(0)

## Second Image

In [ ]:
contours, hierarchy = cv2.findContours(sure_bg,
                                       cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
print("Number of Contours found = " + str(len(contours)))
img = cv2.imread(srcs[1] + ".jpg")

cv2.drawContours(img, contours, -1, (0, 255, 0), 3)

cv2.imshow('Contours', img)
cv2.waitKey(0)

In [ ]:
for contour in contours:
    res = cv2.matchShapes(contoursBase[0], contour, 1, 0)
    print(res)

# Template Matching

In [68]:
#target image
img = cv2.imread('diningbottle.jpg', 0)
img2 = img.copy()
#template image
template = cv2.imread('bottlenobg.jpeg', 0)
width, height = template.shape[::-1]
w_s, h_s = img.shape[::-1]
if width > w_s:
    width = w_s
    height = int(height*w_s/width)
    print("width")
if height > h_s:
    height = h_s
    width = int(width*h_s/height)
    print("height")

dim = (width, height)
print("(", width, ",", height, ")")
template = cv2.resize(template, dim, interpolation = cv2.INTER_AREA)

( 880 , 1087 )


In [70]:
# All the 6 methods for comparison in a list
methods = ['cv2.TM_CCOEFF', 'cv2.TM_CCOEFF_NORMED', 'cv2.TM_CCORR',
           'cv2.TM_CCORR_NORMED', 'cv2.TM_SQDIFF', 'cv2.TM_SQDIFF_NORMED']
t = 0

w, h = template.shape[::-1]
print("(", w, ",", h, ")")
ratio = w_s / h_s

for meth in methods:
    img = img2.copy()
    method = eval(meth)
    # Apply template Matching
    res = cv2.matchTemplate(img, template, method)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
    # If the method is TM_SQDIFF or TM_SQDIFF_NORMED, take minimum
    if method in [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]:
        top_left = min_loc
        bottom_right = max_loc
    else:
        top_left = max_loc
        bottom_right = min_loc
    bottom_right = (top_left[0] + w, top_left[1] + h)
    cv2.rectangle(img, top_left, bottom_right, 255, 2)
    print("top left:", top_left ,"bottom right:", bottom_right)
    #plt.subplot(121), plt.imshow(res, cmap='gray')
    #plt.title('Matching Result'), plt.xticks([]), plt.yticks([])
    #plt.subplot(122), plt.imshow(img, cmap='gray')
    #plt.title('Detected Point'), plt.xticks([]), plt.yticks([])
    #plt.suptitle(meth)
    
    img = cv2.resize(img, (int(ratio*512), 512))
    
    cv2.imshow("technique: "+ meth, img)
    
    t += 1

    #plt.show()

cv2.waitKey(0)

( 880 , 1087 )
top left: (556, 0) bottom right: (1436, 1087)
top left: (556, 0) bottom right: (1436, 1087)
top left: (0, 0) bottom right: (880, 1087)
top left: (556, 0) bottom right: (1436, 1087)
top left: (556, 0) bottom right: (1436, 1087)
top left: (556, 0) bottom right: (1436, 1087)


-1